In [ ]:
import cv2
import mediapipe as mp
import numpy as np

In [ ]:
# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True)

In [ ]:
# Load sunglasses image with alpha channel
sunglasses = cv2.imread("sunglass2.png", cv2.IMREAD_UNCHANGED)  # To retain the alpha channel transparency

# Validate sunglasses image
if sunglasses is None:
    raise ValueError("Error: Could not load sunglasses image. Check the file path!")
if sunglasses.shape[-1] != 4:  # Ensure it has an alpha channel
    raise ValueError("Error: Sunglasses image must be a PNG with transparency!")

def overlay_image_alpha(background, overlay, x, y, overlay_size):
    """Overlay an image (with alpha) onto another image at a given position."""
    overlay = cv2.resize(overlay, overlay_size)  # Resize sunglasses

    # Extract the RGBA channels
    b, g, r, a = cv2.split(overlay)

    # Normalize alpha channel to range [0, 1]
    alpha = a.astype(float) / 255.0

    # Get height and width of the overlay image
    h, w = overlay.shape[:2]

    # Ensure overlay does not go out of frame bounds
    if x < 0:
        overlay = overlay[:, -x:]  # Trim left part
        alpha = alpha[:, -x:]
        x = 0
    if y < 0:
        overlay = overlay[-y:, :]
        alpha = alpha[-y:, :]
        y = 0
    if x + w > background.shape[1]:
        overlay = overlay[:, :background.shape[1] - x]
        alpha = alpha[:, :background.shape[1] - x]
    if y + h > background.shape[0]:
        overlay = overlay[:background.shape[0] - y, :]
        alpha = alpha[:background.shape[0] - y]

    # Get ROI from background where overlay will be placed
    roi = background[y:y+h, x:x+w]

    # Apply the alpha blending equation: new_pixel = alpha * overlay_pixel + (1 - alpha) * background_pixel
    for c in range(3):  # Apply blending for each color channel (BGR)
        # Blends the sunglasses with the background using the alpha blending formula: \text{new_pixel} = \alpha \times \text{overlay_pixel} + (1 - \alpha) \times \text{background_pixel}
        roi[:, :, c] = (alpha * overlay[:, :, c] + (1 - alpha) * roi[:, :, c]).astype(np.uint8)

    background[y:y+h, x:x+w] = roi  # Update background with overlaid image

# Open webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Convert frame to RGB for MediaPipe processing
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Get landmark positions for left and right eye
            h, w, _ = frame.shape
            left_eye = face_landmarks.landmark[33]  # Right eye (from viewer's perspective)
            right_eye = face_landmarks.landmark[263]  # Left eye

            # Convert to pixel coordinates
            # Converts normalized landmark coordinates to pixel coordinates.
            left_eye = (int(left_eye.x * w), int(left_eye.y * h))
            right_eye = (int(right_eye.x * w), int(right_eye.y * h))

            # Calculate sunglasses size and position
            width = int(1.65 * (right_eye[0] - left_eye[0]))  # Scale sunglasses width
            height = int(width * sunglasses.shape[0] / sunglasses.shape[1])  # Maintain aspect ratio
            x = left_eye[0] - int(0.2 * width)  # Adjust position
            y = left_eye[1] - int(0.5 * height)

            # Overlay sunglasses with proper transparency handling
            overlay_image_alpha(frame, sunglasses, x, y, (width, height))

    cv2.imshow("Virtual Try-On", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
